In [50]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix


In [42]:
# get the BioGRID_ID_Interactor_A and BioGRID_ID_Interactor_B, combine and create a new csv with them and their 
#      Official_Symbol_Interactor_A Official_Symbol_Interactor_B

# add the essentiality data to the csv according to Official_Symbol_Interactor_A & Official_Symbol_Interactor_B
# create y, add the essentiality data according to the order of the Protein_ID on ppiemb.csv 

In [51]:
# ids_df =  pd.read_csv("BIOGRID-ORGANISM-Saccharomyces_cerevisiae.csv", usecols= ['BioGRID ID Interactor A', 'BioGRID ID Interactor B', 'Systematic Name Interactor A', 'Systematic Name Interactor B' ]) 
a_ids_df =  pd.read_csv("BIOGRID-ORGANISM-Saccharomyces_cerevisiae.csv", usecols= ['BioGRID ID Interactor A', 'Systematic Name Interactor A']) 
b_ids_df =  pd.read_csv("BIOGRID-ORGANISM-Saccharomyces_cerevisiae.csv", usecols= ['BioGRID ID Interactor B', 'Systematic Name Interactor B']) 

a_ids_df.head()

,BioGRID ID Interactor A,Systematic Name Interactor A
0,31676,YLR418C
1,34272,YOL145C
2,31676,YLR418C
3,34518,YOR123C
4,31676,YLR418C


In [52]:
b_ids_df.head()

,BioGRID ID Interactor B,Systematic Name Interactor B
0,34272,YOL145C
1,31676,YLR418C
2,34518,YOR123C
3,31676,YLR418C
4,32973,YBR279W


In [53]:
a_ids_df.count()

BioGRID ID Interactor A         783485
Systematic Name Interactor A    783485
dtype: int64

In [54]:
b_ids_df.count()
b_ids_df.drop_duplicates

BioGRID ID Interactor B         783485
Systematic Name Interactor B    783485
dtype: int64

In [55]:
ids_emb_df = pd.read_csv("ppiemb.csv", usecols=["Protein_ID"])
ids_emb_df.head()

,Protein_ID
0,33604
1,32517
2,33032
3,32122
4,31903


In [74]:
a_ids_df = a_ids_df.rename(columns = {'BioGRID ID Interactor A':'Protein_ID'})
b_ids_df = b_ids_df.rename(columns = {'BioGRID ID Interactor B':'Protein_ID'})

interactors_df = pd.merge(a_ids_df, b_ids_df, on=['Protein_ID'], how='inner')
interactors_df.head()



,Protein_ID,Systematic Name Interactor A,Systematic Name Interactor B
0,31676,YLR418C,YLR418C
1,31676,YLR418C,YLR418C
2,31676,YLR418C,YLR418C
3,31676,YLR418C,YLR418C
4,31676,YLR418C,YLR418C


In [75]:
interactors_df.drop(['Systematic Name Interactor B'], axis=1, inplace=True)
interactors_df = interactors_df.rename(columns = {'Systematic Name Interactor A':'Official_Symbol_Interactor'})
interactors_df.head()

,Protein_ID,Official_Symbol_Interactor
0,31676,YLR418C
1,31676,YLR418C
2,31676,YLR418C
3,31676,YLR418C
4,31676,YLR418C


In [68]:
interactors_df['Protein_ID'].unique()
interactors_df.count()

Protein_ID                    5756
Official_Symbol_Interactor    5756
dtype: int64

In [31]:
ids_emb_df.head()

,Protein_ID
0,33604
1,32517
2,33032
3,32122
4,31903


In [69]:
ids_emb_df.count()

Protein_ID    7349
dtype: int64

In [72]:
ids_emb_df['Protein_ID'].unique()

array([  33604,   32517,   33032, ..., 1258635,  276279,   31678],
      dtype=int64)

In [73]:
ids_emb_df.drop_duplicates()
ids_emb_df.count()

Protein_ID    7349
dtype: int64

In [39]:
# interactors_df = pd.merge(a_interactors_df, b_interactors_df, on=['Protein_ID'], how='inner').drop_duplicates()

interactors_df = pd.merge(a_interactors_df, b_interactors_df, on=['Protein_ID'], how='inner')

interactors_df.head()

,Protein_ID,Systematic Name Interactor A,Systematic Name Interactor B
0,31676,YLR418C,YLR418C
1,34272,YOL145C,YOL145C
2,34518,YOR123C,YOR123C
3,32973,YBR279W,YBR279W
4,35097,YML069W,YML069W


In [40]:
interactors_df.count()

Protein_ID                      5756
Systematic Name Interactor A    5756
Systematic Name Interactor B    5756
dtype: int64

In [56]:
# get the Official_Symbol_Interactor on esn amd match it with the Official Symbol Interactor A & Official Symbol Interactor B from ppiemb

# DF[DF['ALPHABET'].apply(lambda set: any(s in set for s in strings_list))]
